In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import json

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import models.vqvae as vqvae
import utils.losses as losses 
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_VQ, dataset_TM_eval
import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from utils.word_vectorizer import WordVectorizer
from exit.VQVAE_transformer import VQVAETransformer
from exit.motiontransformer import MotionTransformerOnly2, generate_src_mask

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
w_vectorizer = WordVectorizer('./glove', 'our_vab')

if args.dataname == 'kit' : 
    dataset_opt_path = './checkpoints/kit/Comp_v6_KLD005/opt.txt'  
    args.nb_joints = 21
else :
    dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'
    args.nb_joints = 22
wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt
Reading ./checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [4]:
args.batch_size = 8 #256
args.down_t = 2
args.vqvae_transformer = True 
args.window_size = 64
args.window_size = args.window_size if not args.vqvae_transformer else -1
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        num_workers = 1,
                                        unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

100%|██████████| 4888/4888 [00:01<00:00, 3637.42it/s]

Total number of motions 4400


In [5]:
args.nb_code = 512 # 8192 # 
args.code_dim = 128 #512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None

args.quantizer = 'ema_reset'
args.mu = 0.99
args.recons_loss = 'l1_smooth'

if args.vqvae_transformer:
    net = VQVAETransformer(args).cuda()
    Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints, True)
else:
    net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                        args.nb_code,
                        args.code_dim,
                        args.output_emb_width,
                        args.down_t,
                        args.stride_t,
                        args.width,
                        args.depth,
                        args.dilation_growth_rate,
                        args.vq_act,
                        args.vq_norm)
    Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints)
net = torch.nn.DataParallel(net)

args.resume_pth = '/home/epinyoan/git/MaskText2Motion/T2M-BiDirection/output/VQT_1_4ly_emb256_cbEmb128/net_last.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)

net.train()
net.cuda()
''

''

In [6]:
from exit.utils import get_model, visualize_2motions
net.eval()
for gt_motion, m_length in train_loader_iter:
    gt_motion = gt_motion.cuda().float() # (bs, 64, dim)
    src_mask = generate_src_mask(gt_motion.shape[1], m_length).to(gt_motion.device).unsqueeze(-1)
    gt_motion = gt_motion * src_mask
    # code_idx = get_model(net).encode(gt_motion, src_mask)
    
    x_recon, perplexity, z, z_q = get_model(net)(gt_motion, src_mask)
    break

In [13]:
x = gt_motion[0].detach().cpu().numpy()
y = x_recon[0].detach().cpu().numpy()
visualize_2motions(x, train_loader.dataset.std, train_loader.dataset.mean, args.dataname, m_length[0], y)

In [24]:
args.commit = 0.02
args.loss_vel = 0.5

gt_motion, m_length = next(train_loader_iter)
gt_motion = gt_motion.cuda().float() # (bs, 64, dim)

if args.vqvae_transformer:
    src_mask = generate_src_mask(gt_motion.shape[1], m_length).to(gt_motion.device).unsqueeze(-1)
    gt_motion = gt_motion * src_mask
    pred_motion, perplexity, z, z_q = net(gt_motion, src_mask)
    perplexity = perplexity.mean()
    ## lifted from quantize_cnn to calculate perplexity across all batch
    # code_count = code_onehot.sum(dim=-1)
    # prob = code_count / torch.sum(code_count)  
    # perplexity = torch.exp(-torch.sum(prob * torch.log(prob + 1e-7)))

    loss_commit = (z**2 - z_q.detach()**2).sum() / (src_mask.sum() * z.shape[-1])
    loss_vel = Loss.forward_vel(pred_motion, gt_motion, src_mask)
    loss_motion = Loss(pred_motion, gt_motion, src_mask)
else:
    pred_motion, loss_commit, perplexity = net(gt_motion)
    loss_vel = Loss.forward_vel(pred_motion, gt_motion)
    loss_motion = Loss(pred_motion, gt_motion)

loss = loss_motion + args.commit * loss_commit + args.loss_vel * loss_vel

In [25]:
loss_motion, loss_commit, loss_vel, perplexity, prob.shape

(tensor(0.7637, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(8.7131e-06, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(0.3592, device='cuda:0', grad_fn=<DivBackward0>),
 tensor([30.8162,  7.3679, 10.4846, 20.9441], device='cuda:0'),
 torch.Size([2048]))

### Eval

In [ ]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False,
                                        32,
                                        w_vectorizer,
                                        unit_length=2**args.down_t)

In [ ]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

In [ ]:
args.out_dir = './'
a = eval_trans.evaluation_vqvae(
    args.out_dir, 
    val_loader, 
    net, 
    logger, 
    writer, 
    nb_iter = 1, 
    best_fid = 1, 
    best_iter = 1, 
    best_div = 1, 
    best_top1 = 1, 
    best_top2 = 1, 
    best_top3 = 1, 
    best_matching = 1, 
    eval_wrapper=eval_wrapper)


In [ ]:
word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = next(iter(val_loader))
motion.shape

### PoseFormer

In [21]:
from exit.motiontransformer import MotionTransformerOnly2, generate_src_mask
encoder = MotionTransformerOnly2(input_feats=251, 
                                            output_feats=32, 
                                            latent_dim=256, 
                                            num_layers=8).cuda()

In [22]:
motions = torch.rand(2, 196, 251).cuda()
B, N, _ = motions.shape
num_heads = 8
length = torch.tensor([50, 80]).cuda()
src_mask = generate_src_mask(motions.shape[1], length).to(motions.device).unsqueeze(-1)
src_mask_attn = src_mask.view(B, 1, 1, N).repeat(1, num_heads, N, 1)
encoder(motions, src_mask=src_mask_attn).shape
src_mask.shape

torch.Size([2, 196, 1])